In [1]:
from os import chdir
chdir('/home/jovyan')

In [2]:
from lib.postgres import connect_to_postgres

In [3]:
con, cur = connect_to_postgres()
cur.execute("""
BEGIN;
ALTER TABLE adult ADD COLUMN _id SERIAL PRIMARY KEY;
ALTER TABLE adult ADD COLUMN target BOOLEAN;
COMMIT;
""")
con.close()

In [4]:
con, cur = connect_to_postgres()
cur.execute("""SELECT DISTINCT(income_label) FROM adult;""")
print(cur.fetchall())
con.close()

[(' <=50K',), (' >50K',)]


In [5]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label FROM adult WHERE target is NULL;""")
this_id,income_label = cur.fetchone()

greater_than_50k = (income_label == ' >50K')

cur.execute("""
BEGIN;
UPDATE adult
SET target = {}
WHERE _id = {};
COMMIT;
""".format(greater_than_50k,this_id))

con.close()

In [6]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label, target FROM adult WHERE _id = {};
""".format(this_id))
print(this_id,cur.fetchone())
con.close()

1 (1, ' <=50K', False)


In [7]:
def encode_target(_id):
    con, cur = connect_to_postgres()
    cur.execute("""SELECT _id, income_label FROM adult WHERE _id = {};
    """.format(_id))
    this_id,income_label = cur.fetchone()
    assert this_id == _id
    
    greater_than_50k = (income_label == ' >50K')
    
    cur.execute("""
        BEGIN;
        UPDATE adult
        SET target = {}
        WHERE _id = {};
        COMMIT;
        """.format(greater_than_50k,_id))

    con.close()
    

In [20]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id FROM adult WHERE target is NULL;""")
this_id, = cur.fetchone()
encode_target(this_id)
con.close()

In [21]:
con, cur = connect_to_postgres()
cur.execute("""SELECT _id, income_label, target FROM adult WHERE _id = {};
""".format(this_id))
print(this_id,cur.fetchone())
con.close()

8 (8, ' >50K', True)
